In [7]:
import os
import tensorflow
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import pandas as pd
from termcolor import colored



Using TensorFlow backend.


In [8]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [9]:

# Import twitter data
print(colored("Loading train and test data", "green"))

tr = pd.read_csv("/content/drive/My Drive/big.csv")
te = pd.read_csv("/content/drive/My Drive/twt_#ifc.csv")

tr.full_text=tr.full_text.astype(str)
te.full_text=te.full_text.astype(str)
print(colored("... \nData loaded", "green"))


Loading train and test data
... 
Data loaded


In [10]:

# Tokenization
print(colored("Tokenizing and padding data", "green"))
tokenizer = Tokenizer(num_words = 2000, split = ' ')
tokenizer.fit_on_texts(tr['full_text'].astype(str).values)
train_tweets = tokenizer.texts_to_sequences(tr['full_text'].astype(str).values)
max_len = max([len(i) for i in train_tweets])
train_tweets = pad_sequences(train_tweets, maxlen = max_len)
test_tweets = tokenizer.texts_to_sequences(te['full_text'].astype(str).values)
test_tweets = pad_sequences(test_tweets, maxlen = max_len)
print(colored("Tokenizing and padding complete", "yellow"))

Tokenizing and padding data
Tokenizing and padding complete


In [14]:
# Building the model
print(colored("Creating the LSTM model", "yellow"))
model = Sequential()
model.add(Embedding(2000, 128, input_length = train_tweets.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
model.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
model.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
model.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = True))
model.add(LSTM(256, dropout = 0.2,recurrent_dropout = 0.5,return_sequences = False))

model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Creating the LSTM model
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 37, 128)           256000    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 37, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 37, 256)           394240    
_________________________________________________________________
lstm_5 (LSTM)                (None, 37, 256)           525312    
_________________________________________________________________
lstm_6 (LSTM)                (None, 37, 256)           525312    
_________________________________________________________________
lstm_7 (LSTM)                (None, 37, 256)           525312    
_________________________________________________________________
lstm_8 (LSTM)                (

In [16]:
# Training the model
print(colored("Training the LSTM model", "green"))
history = model.fit(train_tweets, pd.get_dummies(tr['sent_score']).values, epochs = 3, batch_size = 128)
print(colored(history, "green"))

Training the LSTM model
Epoch 1/3
20741/20741 [==============================] - 340s 16ms/step - loss: 0.7144 - accuracy: 0.6756
Epoch 2/3
20741/20741 [==============================] - 335s 16ms/step - loss: 0.6182 - accuracy: 0.7388
Epoch 3/3
20741/20741 [==============================] - 342s 16ms/step - loss: 0.5691 - accuracy: 0.7625


In [17]:
 print(colored("Testing the LSTM model", "green"))

loss, accuracy = model.evaluate(train_tweets, pd.get_dummies(tr['sent_score']).values, batch_size = 128)
print(colored("Train accuracy: {}".format(accuracy),"yellow"))

score, accuracy = model.evaluate(test_tweets, pd.get_dummies(te['sent_score']).values, batch_size = 128)
print(colored("Test accuracy: {}".format(accuracy),"yellow"))



Testing the LSTM model
20741/20741 [==============================] - 96s 5ms/step
Train accuracy: 0.8037703037261963
1749/1749 [==============================] - 8s 5ms/step
Test accuracy: 0.7615780234336853
